In [ ]:
!pip install sentencepiece transformers
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.1 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
# Summarize an Article
url = 'https://finance.yahoo.com/news/adani-slides-report-u-regulatory-014139680.html?.tsrc=fin-srch'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [ ]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')
ARTICLE = ' '.join(words)

summary = summarizer(ARTICLE, max_length=200, min_length=50, do_sample=False)[0]['summary_text']

Your max_length is set to 200, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


In [ ]:
print(summary)

"Our engineers are working quickly to resolve the issue. Thank you for your patience," the company said in a statement. "We are working on a fix for the issue," it added. "The issue has been resolved and we are looking forward to moving on with our lives"


In [ ]:
# Building a PipeLine for News Summarization
monitored_ticker = ['hello']

In [ ]:
# Search for news on Yahoo Finance
def search_for_stock_news(ticker):
    search_url = f'https://www.google.com/search?q=india+stock+news&tbm=nws'
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [ ]:
# Need to Clean these Links only select the important ones
raw_urls = {ticker: search_for_stock_news(ticker) for ticker in monitored_ticker}

In [ ]:
# Strip out unwanted URL's
import re

exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [ ]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [ ]:
cleaned_urls = {ticker: strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_ticker}

In [ ]:
print(cleaned_urls)

{'hello': ['https://mintgenie.livemint.com/news/markets/info-edge-india-share-price-today-live-updates-info-edge-india-stock-plunges-in-trading-today-151692673236234', 'https://www.moneycontrol.com/news/business/markets/stock-market-live-sensex-nifty-50-share-price-gift-nifty-latest-updates-22-08-2023-11233431.html', 'https://mintgenie.livemint.com/news/markets/affle-india-share-price-today-live-updates-affle-india-stock-plummets-in-trading-today-151692673940048', 'https://www.livemint.com/market/stock-market-news/day-trading-guide-for-today-six-stocks-to-buy-or-sell-on-monday-21st-august-11692587583015.html', 'https://mintgenie.livemint.com/news/markets/cummins-india-share-price-today-live-updates-cummins-india-stock-surges-in-trading-151692674213411', 'https://www.livemint.com/market/live-blog/coal-india-share-price-live-blog-for-21-aug-2023-11692586019617.html', 'https://www.google.com/search?q%3Dindia%2Bstock%2Bnews%26tbm%3Dnws%26pccc%3D1', 'https://mintgenie.livemint.com/news/mark

In [ ]:
# Search And Scrape Cleaned Urls
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')
        if 200 < len(words):
          if len(words) > 1024:
            words = words[:1020]
          ARTICLE = ' '.join(words)
          ARTICLES.append(ARTICLE)
    return ARTICLES

In [ ]:
articles = {ticker: scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_ticker}

In [ ]:
# Summarize all articles
def summarize(articles):
    summaries = []
    for article in tqdm(articles):
        try:
          summary = summarizer(article, max_length=200, min_length=50, do_sample=False)[0]['summary_text']
        except IndexError:
          continue
        summaries.append(summary)
    return summaries

In [ ]:
summaries = {ticker: summarize(articles[ticker]) for ticker in monitored_ticker}

100%|██████████| 5/5 [01:41<00:00, 20.34s/it]


In [ ]:
print(summaries)

{'hello': ['KV Kamath, Non-executive chairman of JFSL says at listing ceremony. Lenovo IdeaPad Slim 3 Intel Core i3-1115G4 11th Gen 15.6" (39.62cm) FHD Laptop (8GB/256GB SSD/Win 11/Office 2021/3 Month Game Pass/Arctic Grey/1.65Kg), 82H803TXIN 29% OFF IFB 30 L Convection Microwave Oven (30SC4, Metallic Silver, with Starter Kit) 13% OFF Philips Ac1215/20 Air Purifier, Long Hepa Filter Life Upto 17000 Hours, Removes 99.97% Airborne Pollutants. 20% OFF HUL Pureit Eco Water Saver Mineral RO+UV+MF AS wall mounted/Counter top Black 10L Water Purifier 44% OFF GoPro HERO9 Action Camera with Free Enduro Rechargeable Battery - Waterproof Action', 'Lenovo IdeaPad Slim 3 Intel Core i3-1115G4 11th Gen 15.6" (39.62cm) FHD Laptop (8GB/256GB SSD/Win 11/Office 2021/3 Month Game Pass/Arctic Grey/1.65Kg), 82H803TXIN 29% OFF IFB 30 L Convection Microwave Oven (30SC4, Metallic Silver, with Starter Kit) 13% OFF Philips Ac1215/20 Air Purifier, Long Hepa Filter Life Upto 17000 Hours, Removes 99.97% Airborne Po

In [ ]:
# Sentiment Analysis Pipeline
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Sigma/financial-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("Sigma/financial-sentiment-analysis")


sentiment = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
scores = {ticker: sentiment(summaries[ticker]) for ticker in monitored_ticker}
scores

{'hello': [{'label': 'LABEL_1', 'score': 0.9970318078994751},
  {'label': 'LABEL_1', 'score': 0.9966624975204468}]}

In [ ]:
# Exporting Results as a CSV
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_ticker:
        for counter in range(len(summaries[ticker])):
            output_this=[
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [ ]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['HDFC BANK',
  '37% OFF Samsung Galaxy M34 5G (Prism Silver, 8GB, 128GB Storage) 19% OFF Lenovo IdeaPad Slim 3 Intel Core i3 11th Gen 14" (35.56cm) FHD Thin & Light Laptop (8GB/512GB SSD/Windows 11/Office 2021/2Yr warranty/3months Game Pass/Arctic Grey/1.41Kg), 82H701DNIN 41% OFF IFB 25 L Solo Microwave Oven (25PM2S, Silver) 8% OFF Philips Ac1215/20 Air Purifier, Long Hepa Filter Life Upto 17000 Hours, Removes 99.97% Airborne Pollutants.',
  'LABEL_1',
  0.9970038533210754,
  'https://www.livemint.com/market/stock-market-news/from-hdfc-bank-to-coforge-yes-securities-recommends-10-stocks-to-buy-for-august-11691424779958.html'],
 ['HDFC BANK',
  'Nifty futures traded 17 points, or 0.09 per cent, lower at 19,664.50, signaling that Dalal Street was headed for a flat start on Tuesday. PSU Banks: Shares of state-run banks are likely to be in focus after the aggregate net profit more than doubled to Rs 34,418 crore in Q1.',
  'LABEL_0',
  0.9939054846763611,
  'https://www.livemint.com/comp

In [ ]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])
# LABEL_2 = Positive
# LABEL_1 = Negative
# LABEL_0 = Neutral

In [ ]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)